<a href="https://colab.research.google.com/github/usaginaritai/all-progame/blob/main/%E9%96%8B%E7%99%BA%E3%81%BE%E3%81%A8%E3%82%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

１　おみくじシステム（画像のCHOOSEボタンを押すと、ランダムに大吉、中吉、小吉、凶が表せることができる）

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <style>
        body {
            background-image: url(./jinjia.png);
            background-size: cover;
            margin: 0;
            padding: 0;
            height: 100vh;
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
        }
        #omikujihako {
            cursor: pointer;
        }
        form {
            margin-top: 20px;
        }
    </style>
</head>
<body>
    <img id="omikujihako" src="omikujihako.png" border="0" name="kuji">
    <form>
        <input type="button" value="choose" onclick="myChange()">
    </form>

    <script type="text/javascript">
        const myImage = [
            "./daikichi.png",
            "./chukichi.png",
            "./shokichi.png",
            "./kyo.png"
        ];

        function myChange() {
            const myRnd = Math.floor(Math.random() * myImage.length);
            document.kuji.src = myImage[myRnd];
        }
    </script>
</body>
</html>


２：マイコンボード上のキチンタイマー（３つファイル）　（stop_w_timer.ino）

In [ ]:
#ifndef _COUNT_H_
#define _COUNT_H_

enum { DISP0, DISP1, DISP2, DISP3, DISP4 };
enum { NUM0, NUM1, NUM2, NUM3, NUM4, NUM5, NUM6, NUM7, NUM8, NUM9};
typedef enum { SW0, SW1 } sw_t;
typedef enum { SW_OFF, SW_PUSH, SW_ON, SW_RELEASE } sw_r;

void  disp_init(void);
void  disp_clear(void);
void  disp_cmd(int cmd);
void  disp_data(int disp, int data);
void  disp_num(int disp, int num);
void  count_up(sw_r sw1);
void  count_down(sw_r sw0);
void  count_s_up_cheak(sw_r sw1);
void  count_s_up(sw_r sw1);
void  count_s_down_cheak(sw_r sw0);
void  count_s_down(sw_r sw0);
void  count_reset(sw_r sw0, sw_r sw1);
void  sw_init(void);
sw_r  sw_read(sw_t sw);

#endif


stop_w_t_count.h

In [ ]:
#include "hm2count.h"

static int disp0 = DISP0;
static int disp1 = DISP1;
static int disp2 = DISP2;
static int disp3 = DISP3;
static int disp4 = DISP4;
static int num0 = NUM0;
static int num1 = NUM0;
static int num2 = NUM0;
static int num3 = NUM0;

void setup() {
  sw_init();
  disp_init();
}

void loop() {
  sw_r sw0 = sw_read(SW0);
  sw_r sw1 = sw_read(SW1);

  /*ストップウォッチ*/
  st_watch_up(sw0);

  /*リセット*/
  st_watch_reset(sw0 , sw1);

  /*長押ししたらカウントダン*/
    st_watch_down(sw0);

  count_up(sw1);

}


stop_w_t_count.ino

In [ ]:
#include <Wire.h>
#include "hm2count.h"

#define DISP_I2C_ADRS  0x70
#define NUM_I2C_ADRS 0x70

static const int disp_adrs[] = { 0x08, 0x06, 0x02, 0x00, 0x04 };
static const int num_adrs[] = {0x3F, 0x06, 0x5b, 0x4F, 0x66,0x6D, 0x7D, 0x27, 0x7F, 0x6F};

////////////////////////////////////////////////////////////
// void disp_init(void)
// [機能] 7セグメント表示器の初期設定を行う
// [引数] なし
// [戻り値]  なし

void disp_init(void) {
  Wire.begin(DISP_I2C_ADRS);
  // system setup  : normal operation mode システム設定 通常モード
  disp_cmd(0x21);
  // dimming set   : 16/16 duty             調光セット16/16デゥーティ
  disp_cmd(0xEF);
  // display setup : display on / blinking off  表示設定 表示音/点滅オフ
  disp_cmd(0x81);
  disp_clear();
  //数字の表示。初期状態
  disp_num(disp0, num0);
  disp_num(disp1, num1);
  disp_num(disp2, num2);
  disp_num(disp3, num3);
  disp_data(disp4, 0x03);
}

////////////////////////////////////////////////////////////
// void disp_init(void)
// [機能] 7セグメント表示器の表示内容をすべて消去する
// [引数] なし
// [戻り値]  なし

void disp_clear(void) {
  Wire.beginTransmission(DISP_I2C_ADRS);
  Wire.write(0x00);
  for (int adrs = 0x00; adrs <= 0x0F; adrs++) {
    Wire.write(0x00);
  }
  Wire.endTransmission();
}

////////////////////////////////////////////////////////////
// void disp_cmd(int cmd)
// [機能] 7セグメント表示器に命令cmdを送る
// [引数] cmd 命令を表す1byteの値
// [戻り値]  なし

void disp_cmd(int cmd) {
  Wire.beginTransmission(DISP_I2C_ADRS);
  Wire.write(cmd);
  Wire.endTransmission();
}

////////////////////////////////////////////////////////////
// void disp_data(int disp, int data)
// [機能] 7セグメント表示器のdisp桁目にdataを表示する
// [引数] disp  表示位置を表す値
//      DISP0,DISP1,DISP2,DISP3,DISP4のいずれか
//    data  表示する1byteの値
// [戻り値]  なし

void disp_data(int disp, int data) {
  if (disp < DISP0 || disp > DISP4) {
    return;
  }
  Wire.beginTransmission(DISP_I2C_ADRS);
  Wire.write(disp_adrs[disp]);
  Wire.write(data);
  Wire.endTransmission();
}

void disp_num(int disp, int num) {
  //dispに関するエラー処理
  if (disp < DISP0 || disp > DISP3) {
    return;
  }
  //numに関するエラー処理
  if (num < NUM0 || num > NUM9) {
    return;
  }
  /*Wire.beginTransmission
    → I2Cスレーブに対して送信処理を始める*/
  Wire.beginTransmission(DISP_I2C_ADRS);
  Wire.beginTransmission(NUM_I2C_ADRS);
  /*Wire.write
    → データを送信するときに使う*/
  Wire.write(disp_adrs[disp]);
  Wire.write(num_adrs[num]);
  /*Wire.endTransmission
    → 送信の完了*/
  Wire.endTransmission();
}

#define PIN_SW0  A0
#define PIN_SW1 A1

typedef enum { ST_OFF, ST_ON } st_t;
static st_t state[] = { ST_OFF, ST_OFF };

////////////////////////////////////////////////////////////
// void sw_init(void)
// [機能] 押しボタンスイッチの初期設定を行う
// [引数] なし
// [戻り値]  なし

void sw_init(void) {
  pinMode(PIN_SW0, INPUT);
  pinMode(PIN_SW1, INPUT);
}

////////////////////////////////////////////////////////////
// sw_r sw_read(sw_t sw)
// [機能] 引数で指定された押しボタンスイッチの状態を読み込む
// [引数] sw  押しボタンスイッチを表す値
//      SW0,SW1のいずれか
// [戻り値]  押しボタンスイッチの状態を表す値
//    SW_OFF,SW_PUSH,SW_ON,SW_RELEASEのいずれか

sw_r sw_read(sw_t sw) {
  int val;
  if (sw == SW0) {
    val = digitalRead(PIN_SW0);
  } else if (sw == SW1) {
    val = digitalRead(PIN_SW1);
  } else {
    return SW_OFF;    // wrong argument
  }
  if (state[sw] == ST_OFF) {
    if (val == LOW) {
      return SW_OFF;
    } else {
      state[sw] = ST_ON;
      return SW_PUSH;
    }
  } else {
    if (val == LOW) {
      state[sw] = ST_OFF;
      return SW_RELEASE;
    } else {
      return SW_ON;
    }
  }
}

////////////////////////////////////////////////////////////
// void  sw_watch_up(sw_r sw0)
// [機能] ストップタイマのカウントアップ
// [引数] sw0　sw0が押したらカウントアップ実行する
// [戻り値]  なし
void  st_watch_up(sw_r sw0) {
  if (sw0 == SW_PUSH) {
    while (1) {　//無限ループを使って、カウントアップ機能を実行する
      if (num0 == NUM9 && num1 == NUM9 && num2 == NUM9 && num3 == NUM9) {//前回課題から引用したカウントアップする条件
        disp_num(disp0, num0);
        disp_num(disp1, num1);
        disp_num(disp2, num2);
        disp_num(disp3, num3);
        return;
      }

      num0 = (num0 == NUM9) ? NUM0 : num0 + 1;
      disp_num(disp0, num0);

      if (num0 == NUM0) {
        num1 = (num1 == NUM9) ? NUM0 : num1 + 1;
        disp_num(disp1, num1);

        if (num1 == NUM0) {
          num2 = (num2 == NUM9) ? NUM0 : num2 + 1;
          disp_num(disp2, num2);

          if (num2 == NUM0) {
            num3 = (num3 == NUM9) ? NUM0 : num3 + 1;
            disp_num(disp3, num3);
          }
        }
      }
      delay (10);//チャタリングキャンセル兼10msecごとに数を増やす
      sw_r sw0 = sw_read(SW0);
      if (sw0 == SW_PUSH) {//ボタンをもう一度押したらループを抜ける
        break;
      }
    }
  }
}

////////////////////////////////////////////////////////////
// void st_watch_reset(sw_r sw0 , sw_r sw1)
// [機能] リセット機能
// [引数] sw0　sw1　sw0とsw1両方が押したらリセットする
// [戻り値]  なし
void st_watch_reset(sw_r sw0 , sw_r sw1) {
  if (sw0 == SW_ON && sw1 == SW_ON) {
    num0 = NUM0;
    num1 = NUM0;
    num2 = NUM0;
    num3 = NUM0;
    disp_num(disp0, num0);
    disp_num(disp1, num1);
    disp_num(disp2, num2);
    disp_num(disp3, num3);
    delay (10);
  }
}

////////////////////////////////////////////////////////////
// st_watch_down(sw_r sw0)
// [機能] カウントダン機能、キーリピート機能
// [引数] sw0　sw0が押したらカウントダンするする
// [戻り値]  なし
void  st_watch_down(sw_r sw0) {//sw0がONの時に起動。
  if (sw0 == SW_PUSH) {
    while (1) {
      if (num0 == NUM0 && num1 == NUM0 && num2 == NUM0 && num3 == NUM0) {
        disp_num(disp0, num0);
      } else {
        num0 = (num0 == NUM0) ? NUM9 : num0 - 1;
        disp_num(disp0, num0);
      }

      if (num0 == NUM9) {
        if (num1 == NUM0 && num2 == NUM0 && num3 == NUM0) {
          disp_num(disp1, num1);
          num0 = NUM9;
        } else {
          num1 = (num1 == NUM0) ? NUM9 : num1 - 1;
          disp_num(disp1, num1);
        }
      }

      if (num1 == NUM9 && num0 == NUM9) {
        if (num2 == NUM0 && num3 == NUM0) {
          disp_num(disp2, num2);
          num1 = NUM9;
        } else {
          num2 = (num2 == NUM0) ? NUM9 : num2 - 1;
          disp_num(disp2, num2);
        }
      }

      if (num2 == NUM9 && num1 == NUM9 && num0 == NUM9) {
        if (num3 == NUM0) {
          disp_num(disp3, num3);
          num2 = NUM9;
        } else {
          num3 = (num3 == NUM0) ? NUM9 : num3 - 1;
          disp_num(disp3, num3);
        }
      }
      delay (10);
      sw_r sw1 = sw_read(SW1);
      if (sw1 == SW_PUSH) {
        break;
      }
    }
  }
}

////////////////////////////////////////////////////////////
// void  count_up(sw_r sw1)
// [機能] 開始時間の選択機能、始めたい時間から開始できる
// [引数] sw0　sw0が押したら数字が選択できる
// [戻り値]  なし
void  count_up(sw_r sw1) {

  if (sw1 == SW_PUSH) {
    if (num0 == NUM9 && num1 == NUM9 && num2 == NUM9 && num3 == NUM9) {
      disp_num(disp0, num0);
      disp_num(disp1, num1);
      disp_num(disp2, num2);
      disp_num(disp3, num3);
    } else {
        if (num1 == NUM0) {//100の位が上がるとき
          num2 = (num2 == NUM9) ? NUM0 : num2 + 1;
          disp_num(disp2, num2);

          if (num2 == NUM0) {//1000の位が上がるとき
            num3 = (num3 == NUM9) ? NUM0 : num3 + 1;
            disp_num(disp3, num3);

          }
       }
      }
    }
  }
}

/*count_super_upプログラムの起動用*/
////////////////////////////////////////////////////////////
// void  count_s_up_cheak(sw_r sw1)
// [機能] sw１がONの時、150msec待ち,もう1度sw1を読み込む
// [引数] sw1　sw1がonになった、状態と読み込む
// [戻り値]  なし
void  count_s_up_cheak(sw_r sw1) {
  if (sw1 == SW_ON) {
    //チャタリングキャンセルも兼ねている。
    delay(150);
    sw_r sw1 = sw_read(SW1);//sw1の状態を保持する
    if (sw1 == SW_OFF) {
      return;
    } count_s_up(sw1);//SW_ONの場合、キーリピートを起動
  }
}

////////////////////////////////////////////////////////////
// void  count_s_up(sw_r sw1)
// [機能] 開始時間の選択機能、始めたい時間から開始できる
// [引数] sw１　sw１が押したら数字がダンできる
// [戻り値]  なし
void  count_s_up(sw_r sw1) {
  //sw1がONの時に実行される。
  if (sw1 == SW_ON) {
    if (num0 == NUM9 && num1 == NUM9 && num2 == NUM9 && num3 == NUM9) {
      disp_num(disp0, num0);
      disp_num(disp1, num1);
      disp_num(disp2, num2);
      disp_num(disp3, num3);
      return;
    }

    num0 = (num0 == NUM9) ? NUM0 : num0 + 1;
    disp_num(disp0, num0);
    if (sw1 == SW_OFF) {
    }

    if (num0 == NUM0) {
      num1 = (num1 == NUM9) ? NUM0 : num1 + 1;
      disp_num(disp1, num1);

      if (num1 == NUM0) {
        num2 = (num2 == NUM9) ? NUM0 : num2 + 1;
        disp_num(disp2, num2);

        if (num2 == NUM0) {
          num3 = (num3 == NUM9) ? NUM0 : num3 + 1;
          disp_num(disp3, num3);
        }
      }
    }
  }
}

////////////////////////////////////////////////////////////
// void  count_down(sw_r sw0)
// [機能] 開始時間の選択機能、始めたい時間から開始できる
// [引数] sw１　sw１が押したら数字がダンできる
// [戻り値]  なし
void count_down(sw_r sw0) {
  if (sw0 == SW_PUSH) {
    if (num1 == NUM9 && num0 == NUM9) {//100の位の処理
      if (num2 == NUM0 && num3 == NUM0) {
        disp_num(disp2, num2);
        num1 = NUM9;
      } else {
        num2 = (num2 == NUM0) ? NUM9 : num2 - 1;
        disp_num(disp2, num2);
      }
    }

    if (num2 == NUM9 && num1 == NUM9 && num0 == NUM9) {//1000の位の処理
      if (num3 == NUM0) {
        disp_num(disp3, num3);
        num2 = NUM9;
      } else {
        num3 = (num3 == NUM0) ? NUM9 : num3 - 1;
        disp_num(disp3, num3);
      }
    }
  }
}

/*カウントダウン。キーリピート機能の起動に使う*/
////////////////////////////////////////////////////////////
// count_s_down_cheak(sw_r sw0)
//[機能] sw0がONの時、150msec待ち,もう1度sw1を読み込む
// [引数] sw0　sw0がonになった、状態と読み込む
// [戻り値]  なし
void  count_s_down_cheak(sw_r sw0) {
  if (sw0 == SW_ON) {
    //ONの時、150msec待ちもう1度sw0を読み込む。
    //チャタリングキャンセルも兼ねている。
    delay(150);
    //sw0の状態を保持する
    sw_r sw0 = sw_read(SW0);
    if (sw0 == SW_OFF) {
      return;
      //sw0がONだったらキーリピートを起動
    } count_s_down(sw0);
  }
}


３：来客アラーム（マイコンボードに書き込み）探知センサー前に人が来ると自動的に探知できる、そしてアラームが事前に設定した曲を流します。

In [ ]:
#define SP 12
#define LEG 9
#define EXI 8
#define DO1 261
#define RE1 294
#define MI1 330
#define FA1 349
#define SO1 392
#define RA1 440
#define RA2 466		//ラ#
#define SI1 494

void readreg()		//レジ前センサ
{
    int s;
    while (1)
    {
        s = digitalRead(LEG); // 人感センサを読む
        if (s == HIGH)        // 人がいる
        {
            break;
        }
        delay(500); // 0.5秒待つ
    }
}

void readexi()		//出口センサ
{
    int s;
    while (1)
    {
        s = digitalRead(EXI);	//人感センサを読む
        if (s == HIGH) 			//人がいる
        {
            break;
        }
        delay(500);
    }
}

void humbre(int sim, int sec, int mut)
{
    tone(SP, sim);	//音階の指定
    delay(sec);		//時間の指定
    noTone(SP);		//音を止める
    delay(mut);		//時間の指定
}

void music()
{
    int i;
    int j;

    for (i = 1; i <= 2; i++)
    {
        humbre(DO1, 350, 50);
        humbre(FA1, 250, 50);
        for (j = 1; j <= 3; j++)
        {
            humbre(SO1, 180, 50);
            humbre(FA1, 180, 50);
        }
        humbre(DO1, 300, 50);
        humbre(FA1, 250, 50);
        for (j = 1; j <= 2; j++)
        {
            humbre(SO1, 180, 50);
            humbre(FA1, 180, 50);
        }
        humbre(RA1, 300, 200);
    }
    humbre(DO1, 120, 80);
    humbre(DO1, 120, 80);
    humbre(RE1, 300, 80);
    humbre(FA1, 180, 50);
    humbre(MI1, 300, 100);
    humbre(DO1, 200, 80);
    humbre(RA2, 250, 50);
    humbre(RA1, 350, 80);
    humbre(SO1, 180, 80);
    humbre(FA1, 120, 100);
    humbre(FA1, 300, 200);
    humbre(FA1, 200, 100);
    humbre(SO1, 250, 80);
    humbre(RA1, 300, 80);
    humbre(FA1, 200, 80);
    humbre(SO1, 250, 80);
    humbre(RA1, 300, 100);
    humbre(RA2, 200, 80);
    humbre(RA1, 200, 80);
    humbre(SO1, 200, 80);
    humbre(FA1, 1500, 0);
}

void setup()
{
    pinMode(LEG, INPUT);
    pinMode(EXI, INPUT);

    while (1)
    {
        readreg();
        music();
        readexi();
    }
}
void loop()
{}


４：照明消し忘れ装置（LINUXの仮装マシン上に擬似センサーを構築TCP/IPを利用して、LINEに送信）

簡単な説明:コードは２つある、１つ目は擬似センサコード、２つ目は送信用コード。擬似センサーから０ルーメンから1000ルーメンまで調整できる。ボタンを押しながら、左右移動できる。擬似センサーより、光の強さができ、LINEに送信できる。


実行結果：擬似センサが700ルーメンを超えると、LINEにthe light in this room is onを送る。それ以外の場合はthe light in this room is off


In [ ]:
#!/usr/bin/env python3
#
# Temp sensor server example on Raspberry Pi / VirtualBox
# (Thanks to pythonspot.com!)
# https://pythonspot.com/pyqt5/
#
# References
#
# [1] Qt document
# https://doc.qt.io/qtforpython/PySide2/QtNetwork/QTcpServer.html
#
# [2] PyQt example
# https://github.com/pyqt/examples/blob/_/src/pyqt-official/network/loopback.py
#
#
# Please install PyQt5 before execute
# $ sudo apt update
# $ sudo apt install python3-pyqt5


import sys
from PyQt5 import QtGui
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QSlider, QLabel, QVBoxLayout
from PyQt5.QtGui import QIcon
from PyQt5.QtCore import pyqtSlot, Qt
from PyQt5.QtNetwork import QTcpServer

SERVER_PORT = 8888

class App(QWidget):

    def __init__(self):
        super().__init__()
        self.title = 'Light server'
        self.left = 50
        self.top = 100
        self.width = 400
        self.height = 200
        self.initUI()
        self.server = QTcpServer()
        self.server.listen(port=SERVER_PORT)
        print("Temp Sensor Server started at port ", self.server.serverPort())
        self.server.newConnection.connect(self.acceptConnection)

    def initUI(self):
        self.setWindowTitle(self.title)
        self.setGeometry(self.left, self.top, self.width, self.height)

        self.tlabel = QLabel("Light")
        self.tlabel.setFont(QtGui.QFont("Sanserif", 15))

        self.slider = QSlider(Qt.Horizontal, self)
        self.slider.setToolTip('This is a slider')
        self.slider.setTickInterval(1)
        self.slider.setMinimum(0)
        self.slider.setMaximum(1000)
        self.slider.valueChanged.connect(self.on_slider_changedValue)

        self.label = QLabel("0")
        self.label.setFont(QtGui.QFont("Sanserif", 15))

        self.exit_button = QPushButton('Exit', self)
        self.exit_button.setToolTip('This is an exit button')
        self.exit_button.clicked.connect(self.close)

        layout = QVBoxLayout()
        layout.addWidget(self.tlabel)
        layout.addWidget(self.slider)
        layout.addWidget(self.label)
        layout.addWidget(self.exit_button)

        self.setLayout(layout)
        self.show()

    @pyqtSlot()
    def on_slider_changedValue(self):
        size = self.slider.value()
        self.label.setText(str(size))
        self.show()

    def acceptConnection(self):
        self.tcpServerConnection = self.server.nextPendingConnection()
        self.tcpServerConnection.write(str(self.slider.value()).encode())
        self.tcpServerConnection.close()

app = QApplication(sys.argv)
ex = App()
sys.exit(app.exec_())


In [ ]:
#!/usr/bin/env python3
#
# Temp sensor client example on Raspberry Pi / VirtualBox
#
#


import socket//必要なモジュールをインポートする
import time//必要なモジュールをインポートする
import requests//必要なモジュールをインポートする

SERVER_HOST = '127.0.0.1'//指定したホストのIPアドレス
SERVER_PORT = 8888//接続するサーバーのポート番号
url = "https://notify-api.line.me/api/notify"//notifyのUMLを指定
token = "NdiSmYBnB1ni20u6A7D60tlC0UX10ZbiTK8YfEPyOnW"//トークンを指定
headers = {"Authorization" : "Bearer "+ token}

while True://無限ループ開始
	with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s://使用したソケットオブジェクトを作成
    		s.connect((SERVER_HOST, SERVER_PORT))
    		data = s.recv(128)
    		print(data.decode('ascii'))   //光の数値を表す
	message = data.decode('ascii')//メッセージを送る
	print (message)
	value = int(message)
	print(value)
	if value >= 700://もし700メーメン以上の場合
    		message = "The light in this room is on"
	else://その以外の場合
    		message = "The light in this room is off"
	payload = {"message" :  message}
	r = requests.post(url, headers = headers, params=payload)

	time.sleep(300)//五分一回ループ実行する


５：バスケットボールフェイント動作判定システム（修正すると、サッカーやバスケ以外のスポーツの動作判定できる）

In [ ]:
import numpy as np
from scipy.ndimage import gaussian_filter
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from google.colab import files
import pandas as pd
import csv
import os

# CSVデータ読み込み関数
def read_csv_data(uploaded_files):
    accel_dfs = []
    gyro_dfs = []

    for filename, file_content in uploaded_files.items():
        accel_data = []
        gyro_data = []
        start_recording = False  # センサーデータ記録開始のフラグ

        base_filename = os.path.basename(filename).replace('.csv', '')

        # ファイルを文字列にデコードしてCSVとして読み込む
        file_str = file_content.decode('utf-8').splitlines()
        reader = csv.reader(file_str)
        for row in reader:
            if "sens start" in row[0]:  # 記録開始のキーワードを検出
                start_recording = True
                continue
            if start_recording:
                if row[0] == 'ags':
                    # 加速度データを m/s² に変換
                    accel_x = float(row[2]) * 0.0001 * 9.80665
                    accel_y = float(row[3]) * 0.0001 * 9.80665
                    accel_z = float(row[4]) * 0.0001 * 9.80665
                    accel_data.append([float(row[1]), accel_x, accel_y, accel_z])
                    # ジャイロデータを deg/sec に変換
                    gyro_x = float(row[5]) * 0.01
                    gyro_y = float(row[6]) * 0.01
                    gyro_z = float(row[7]) * 0.01
                    gyro_data.append([float(row[1]), gyro_x, gyro_y, gyro_z])

        # DataFrame に変換
        accel_df = pd.DataFrame(accel_data, columns=['Timestamp', f'{base_filename}_Accel_X', f'{base_filename}_Accel_Y', f'{base_filename}_Accel_Z'])
        gyro_df = pd.DataFrame(gyro_data, columns=['Timestamp', f'{base_filename}_Gyro_X', f'{base_filename}_Gyro_Y', f'{base_filename}_Gyro_Z'])

        accel_dfs.append(accel_df)
        gyro_dfs.append(gyro_df)

    return accel_dfs, gyro_dfs

def plot_multiple_data_with_peaks_and_adjustment(accel_dfs, gyro_dfs, sigma, threshold_factors, manual_adjustment=0, line_basis="max"):
    """
    赤い垂直線の位置を手動で調整し、最大値または最小値を基準として選択可能にする。
    また、赤線と黒線の間にある最小値を強調表示する。
    Y軸およびZ軸において、赤線の後に現れる2番目のピークの位置に黒い線を表示。
    赤線と黒線の間の最小値は黒の実線マーカー、それ以外の最小値は青の実線マーカーで表示。

    :param manual_adjustment: 赤線の手動調整時間（秒）。デフォルトは0。
    :param line_basis: 赤線の基準（最大値 "max" または 最小値 "min"）。
    """
    plt.figure(figsize=(16, 10))
    axes_labels = ['X', 'Y', 'Z']

    for i in range(3):
        plt.subplot(3, 2, 2 * i + 1)  # 加速度データのサブプロット
        for df in accel_dfs:
            min_timestamp = df['Timestamp'].min()
            base_filename = '_'.join(df.columns[1].split('_')[:-2])
            original_data = df[f'{base_filename}_Accel_{axes_labels[i]}']

            # Y軸のみ重力補正を考慮（現状オフセット0）
            if axes_labels[i] == 'Y':
                gravity_offset = 9.8
                corrected_data = original_data - 0
            else:
                corrected_data = original_data

            # ガウス平滑化
            filtered_data = gaussian_filter(corrected_data, sigma=sigma)
            time = (df['Timestamp'] - min_timestamp).values / 1000  # 単位を秒に変換

            # ピーク検出
            peaks, _ = find_peaks(filtered_data)
            threshold = np.std(filtered_data) * threshold_factors[i]
            significant_peaks = peaks[filtered_data[peaks] > threshold]

            # 赤線と黒線の表示位置を決定
            adjusted_time = None
            second_peak_time = None
            if axes_labels[i] in ['Y', 'Z']:
                for gyro_df in gyro_dfs:
                    gyro_min_timestamp = gyro_df['Timestamp'].min()
                    gyro_base_filename = '_'.join(gyro_df.columns[1].split('_')[:-2])
                    gyro_filtered_data = gaussian_filter(gyro_df[f'{gyro_base_filename}_Gyro_Y'], sigma=sigma)
                    gyro_time = (gyro_df['Timestamp'] - gyro_min_timestamp).values / 1000

                    # 基準に従って赤線を描画
                    if line_basis == "max":
                        reference_idx = gyro_filtered_data.argmax()
                    elif line_basis == "min":
                        reference_idx = gyro_filtered_data.argmin()
                    else:
                        raise ValueError("line_basisは'max'または'min'のいずれかで指定してください。")

                    reference_time = gyro_time[reference_idx]
                    adjusted_time = reference_time + manual_adjustment
                    plt.axvline(x=adjusted_time, color='red', linestyle='-', linewidth=3, label=f'赤線位置: {adjusted_time:.2f}s')

                    # 赤線後の2番目のピークに黒線を引く
                    if len(significant_peaks) > 1:
                        post_line_peaks = significant_peaks[time[significant_peaks] > adjusted_time]
                        if len(post_line_peaks) > 1:
                            second_peak_time = time[post_line_peaks[1]]
                            plt.axvline(x=second_peak_time, color='black', linestyle='-', linewidth=3, label=f'2番目のピーク: {second_peak_time:.2f}s')

            # ピーク間の最小値を計算
            if len(significant_peaks) > 1:
                for j in range(len(significant_peaks) - 1):
                    start_idx, end_idx = significant_peaks[j], significant_peaks[j + 1]
                    min_idx = np.argmin(filtered_data[start_idx:end_idx]) + start_idx
                    min_time = time[min_idx]

                    # 赤線と黒線の間かどうかでマーカーの色を変える
                    if adjusted_time is not None and second_peak_time is not None:
                        if adjusted_time < min_time < second_peak_time:
                            plt.scatter(min_time, filtered_data[min_idx], color='black', label='赤黒間の最小値', s=50, zorder=5)
                        else:
                            plt.scatter(min_time, filtered_data[min_idx], color='blue', label='範囲外の最小値', s=50, zorder=5)
                    else:
                        plt.scatter(min_time, filtered_data[min_idx], color='blue', label='範囲外の最小値', s=50, zorder=5)

            # 平滑データとピークをプロット
            plt.plot(time, filtered_data, label=f'平滑化 {base_filename} Accel_{axes_labels[i]}')
            plt.plot(time[significant_peaks], filtered_data[significant_peaks], "o", label=f'有効ピーク > {threshold:.2f}')

        plt.title(f'加速度データ {axes_labels[i]}軸', fontsize=14)
        plt.xlabel('時間 (秒)', fontsize=12)
        plt.ylabel('加速度 ($m/s^2$)', fontsize=12)
        plt.grid(True)

        # ジャイロデータのサブプロット
        plt.subplot(3, 2, 2 * i + 2)
        for gyro_df in gyro_dfs:
            min_timestamp = gyro_df['Timestamp'].min()
            base_filename = '_'.join(gyro_df.columns[1].split('_')[:-2])
            original_data = gyro_df[f'{base_filename}_Gyro_{axes_labels[i]}']
            filtered_data = gaussian_filter(original_data, sigma=sigma)
            time = (gyro_df['Timestamp'] - min_timestamp).values / 1000

            plt.plot(time, filtered_data, label=f'平滑化 {base_filename} Gyro_{axes_labels[i]}')

        plt.title(f'ジャイロデータ {axes_labels[i]}軸', fontsize=14)
        plt.xlabel('時間 (秒)', fontsize=12)
        plt.ylabel('角速度 (deg/sec)', fontsize=12)
        plt.grid(True)

    plt.tight_layout()
    plt.subplots_adjust(hspace=0.5, wspace=0.4)
    plt.show()

# メイン処理の実行部
uploaded_files = files.upload()  # ファイルをアップロード
sigma = 45  # ガウス平滑化のパラメータ
threshold_factors = [1.5, 1.5, 2.0]  # ピーク検出の閾値係数

# 手動調整の入力（赤線を左右にずらす）
manual_adjustment = float(input("赤線の時間シフト量（秒）を入力してください。右にずらす場合は正、左は負（デフォルト0）：") or 0)

# 赤線の基準（最大値か最小値か）を選択
line_basis = input("赤線の基準を選んでください：'max'（最大値）または'min'（最小値）。デフォルトは'max'：").strip().lower() or "max"

if uploaded_files:
    accel_dfs, gyro_dfs = read_csv_data(uploaded_files)
    plot_multiple_data_with_peaks_and_adjustment(accel_dfs, gyro_dfs, sigma, threshold_factors, manual_adjustment, line_basis)


６： バスケットボール、サッカー運動中のステップ識別

In [ ]:
import numpy as np
from scipy.ndimage import gaussian_filter
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from google.colab import files
import pandas as pd
import csv
import os

def read_csv_data(file_list):
    accel_dfs = []
    gyro_dfs = []
    for filename in file_list:
        accel_data = []
        gyro_data = []
        start_recording = False
        base_filename = os.path.basename(filename).replace('.csv', '')

        with open(filename, 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                if "sens start" in row[0]:
                    start_recording = True
                    continue
                if start_recording:
                    if row[0] == 'ags':
                        accel_x = float(row[2]) * 0.0001 * 9.80665
                        accel_y = float(row[3]) * 0.0001 * 9.80665
                        accel_z = float(row[4]) * 0.0001 * 9.80665
                        accel_data.append([float(row[1]), accel_x, accel_y, accel_z])
                        gyro_x = float(row[5]) * 0.01
                        gyro_y = float(row[6]) * 0.01
                        gyro_z = float(row[7]) * 0.01
                        gyro_data.append([float(row[1]), gyro_x, gyro_y, gyro_z])

        accel_df = pd.DataFrame(accel_data, columns=['Timestamp', f'{base_filename}_Accel_X', f'{base_filename}_Accel_Y', f'{base_filename}_Accel_Z'])
        gyro_df = pd.DataFrame(gyro_data, columns=['Timestamp', f'{base_filename}_Gyro_X', f'{base_filename}_Gyro_Y', f'{base_filename}_Gyro_Z'])

        accel_dfs.append(accel_df)
        gyro_dfs.append(gyro_df)

    return accel_dfs, gyro_dfs

def extract_peaks_time(accel_dfs, sigma, threshold_factors):
    y_peak_times = []
    y_peak_values = []
    z_peak_times = []
    z_peak_values = []
    axes_labels = ['X', 'Y', 'Z']

    for i in range(3):
        for df in accel_dfs:
            min_timestamp = df['Timestamp'].min()
            base_filename = '_'.join(df.columns[1].split('_')[:-2])
            original_data = df[f'{base_filename}_Accel_{axes_labels[i]}']

            if axes_labels[i] == 'Y':
                gravity_offset = 9.8
                corrected_data = original_data - gravity_offset
            else:
                corrected_data = original_data

            filtered_data = gaussian_filter(corrected_data, sigma=sigma)
            time = (df['Timestamp'] - min_timestamp).values / 1000

            peaks, _ = find_peaks(filtered_data)
            threshold = np.std(filtered_data) * threshold_factors[i]
            significant_peaks = peaks[filtered_data[peaks] > threshold]

            # 保存峰值时间和值
            if axes_labels[i] == 'Y':
                y_peak_times.extend(time[significant_peaks])
                y_peak_values.extend(filtered_data[significant_peaks])
                print(f"Y-axis peak times for {base_filename}: {time[significant_peaks]}")
            elif axes_labels[i] == 'Z':
                z_peak_times.extend(time[significant_peaks])
                z_peak_values.extend(filtered_data[significant_peaks])
                print(f"Z-axis peak times for {base_filename}: {time[significant_peaks]}")

    matched_peaks = []
    i, j = 0, 0
    while i < len(y_peak_times) and j < len(z_peak_times):
        time_diff = abs(y_peak_times[i] - z_peak_times[j])

        # 初始化递增/递减检测标志为 False
        is_y_decreasing = False
        is_z_increasing = False

        if time_diff < 0.1:
            # 检测成对峰值时，检查是否满足递减与递增条件
            is_y_decreasing = (
                i + 2 < len(y_peak_values) and
                y_peak_values[i] > y_peak_values[i + 1] > y_peak_values[i + 2]
            )
            is_z_increasing = (
                j + 2 < len(z_peak_values) and
                z_peak_values[j] < z_peak_values[j + 1] < z_peak_values[j + 2]
            )

            # 如果 Y 递减且 Z 递增，不记录这对峰值
            if not (is_y_decreasing and is_z_increasing):
                matched_peaks.append((y_peak_times[i], z_peak_times[j]))

            i += 1
            j += 1
        elif y_peak_times[i] < z_peak_times[j]:
            i += 1
        else:
            j += 1

        # 跳过 Y 和 Z 的递减/递增检查范围
        if is_y_decreasing and is_z_increasing:
            i += 3
            j += 3

    print("Matched peak times (Y, Z):", matched_peaks)
    return y_peak_times, z_peak_times, matched_peaks



def plot_multiple_data_with_peaks_and_action(accel_dfs, gyro_dfs, sigma, threshold_factors, matched_peaks):
    plt.figure(figsize=(16, 10))
    axes_labels = ['X', 'Y', 'Z']

    for i in range(3):
        plt.subplot(3, 2, 2*i+1)
        for df in accel_dfs:
            min_timestamp = df['Timestamp'].min()
            base_filename = '_'.join(df.columns[1].split('_')[:-2])
            original_data = df[f'{base_filename}_Accel_{axes_labels[i]}']

            if axes_labels[i] == 'Y':
                gravity_offset = 9.8
                corrected_data = original_data - gravity_offset
            else:
                corrected_data = original_data

            filtered_data = gaussian_filter(corrected_data, sigma=sigma)
            time = (df['Timestamp'] - min_timestamp).values / 1000

            peaks, _ = find_peaks(filtered_data)
            threshold = np.std(filtered_data) * threshold_factors[i]
            significant_peaks = peaks[filtered_data[peaks] > threshold]

            plt.plot(time, filtered_data, label=f'Filtered {base_filename} Accel_{axes_labels[i]}')
            plt.plot(time[significant_peaks], filtered_data[significant_peaks], "o", label="Significant Peaks")

            # 绘制匹配的Y、Z轴峰值的竖线
            for idx, (y_time, z_time) in enumerate(matched_peaks):
                if axes_labels[i] == 'Y':
                    plt.axvline(x=y_time, color='red', linestyle='--')
                    plt.text(y_time + 0.01, max(filtered_data), f'{idx + 1}', color='black', ha='left')
                elif axes_labels[i] == 'Z':
                    plt.axvline(x=z_time, color='red', linestyle='--')
                    plt.text(z_time + 0.01, max(filtered_data), f'{idx + 1}', color='black', ha='left')

            plt.axvline(x=0, color='blue', linestyle='-', linewidth=1.5)
            plt.text(0.01, max(filtered_data) * 0.9, 'START', color='blue', ha='left')

            if significant_peaks.size > 0:
                last_peak_idx = significant_peaks[-1]
                end_idx = last_peak_idx + np.argmin(filtered_data[last_peak_idx:])
                plt.axvline(x=time[end_idx], color='black', linestyle='-', linewidth=1.5)
                plt.text(time[end_idx] + 0.01, min(filtered_data), 'END', color='black', ha='left')

        plt.title(f'Acceleration Data {axes_labels[i]}-Axis', fontsize=14)
        plt.xlabel('Time (seconds)', fontsize=12)
        plt.ylabel('Acceleration ($m/s^2$)', fontsize=12)
        plt.grid(True)

        plt.subplot(3, 2, 2*i+2)
        for df in gyro_dfs:
            base_filename = '_'.join(df.columns[1].split('_')[:-2])
            original_data = df[f'{base_filename}_Gyro_{axes_labels[i]}']
            filtered_data = gaussian_filter(original_data, sigma=sigma)
            time = (df['Timestamp'] - min_timestamp).values / 1000
            plt.plot(time, filtered_data, label=f'Filtered {base_filename} Gyro_{axes_labels[i]}')

        plt.title(f'Gyroscope Data {axes_labels[i]}-Axis', fontsize=14)
        plt.xlabel('Time (seconds)', fontsize=12)
        plt.ylabel('Gyroscope (deg/sec)', fontsize=12)
        plt.grid(True)

    plt.tight_layout()
    plt.subplots_adjust(hspace=0.5, wspace=0.4)
    plt.show()

uploaded_files = files.upload()
sigma = 45
threshold_factors = [2, 0.5, 2.2]
if uploaded_files:
    accel_dfs, gyro_dfs = read_csv_data(uploaded_files)
    y_peak_times, z_peak_times, matched_peaks = extract_peaks_time(accel_dfs, sigma, threshold_factors)
    plot_multiple_data_with_peaks_and_action(accel_dfs, gyro_dfs, sigma, threshold_factors, matched_peaks)

7：FITBIT上に簡単ログインシステム（メインプログラム）

In [ ]:
import { geolocation } from "geolocation";
import clock from "clock";
import * as document from "document";
import {preferences} from "user-settings";

//時計
function zeroPad(i){
    if(i<10){
        i="0"+i;
    }
    return i;
}

clock.granularity="seconds";

//判定座標を記録する
const fixedLatitude = 35.638082;
const fixedLongitude = 139.734504;
//テキスト文字の設定
const myLabel = document.getElementById("myLabel");
const myText = document.getElementById("myText");
const Element = document.getElementById("myElement");
//ボタンの設定
const Button = document.getElementById("button");

myText.text="Group1";
//テキスト文字と時間の表し
clock.ontick = (evt) => {
  let today = evt.date;
  let hours = today.getHours();
  if (preferences.clockDisplay === "12h") {
    hours = hours % 12 || 12;
  } else {
    hours = zeroPad(hours);
  }
  let mins = zeroPad(today.getMinutes());
  let seconds = zeroPad(today.getSeconds());
  myLabel.text = `${hours}:${mins}:${seconds}`;
}

// 現在の居場所確定する
    geolocation.getCurrentPosition(locationSuccess, locationError, {
    timeout: 60 * 1000
    });
    // 位置を処理する
    function locationSuccess(position) {
        console.log(
                    "Latitude: " + position.coords.latitude,
                    "Longitude: " + position.coords.longitude
                    );

        // 今の居場所は固定した場所一致するか判定
        const isInRange = isWithinRange(
                                        position.coords.latitude,
                                        position.coords.longitude
                                        );

        // 成功か失敗か表す
        Button.addEventListener("click", async(evt) => {
            if (isInRange) {
                console.log("Success: login is success");
                // 成功の例
            } else {
                console.log("Failure: login is fall");
                // 失敗の例
            }
        });
            }

function isWithinRange(latitude, longitude) {
  //  Haversine 公式使って距離を計算
  const R = 6371000; // 地球の半径、単位はm
  const lat1 = toRadians(fixedLatitude);
  const lon1 = toRadians(fixedLongitude);
  const lat2 = toRadians(latitude);
  const lon2 = toRadians(longitude);

  const dLat = lat2 - lat1;
  const dLon = lon2 - lon1;

  const a =
    Math.sin(dLat / 2) * Math.sin(dLat / 2) +
    Math.cos(lat1) * Math.cos(lat2) * Math.sin(dLon / 2) * Math.sin(dLon / 2);

  const c = 2 * Math.atan2(Math.sqrt(a), Math.sqrt(1 - a));

  const distance = R * c;

  // 誤差が200メートル以内
  return distance <= 200;
}

// 角度を円弧に変える
function toRadians(degrees) {
  return (degrees * Math.PI) / 180;
}

// エラー発生メッセージ
function locationError(error) {
  console.log("Error: " + error.code, "Message: " + error.message);
}

8：スクリンショットの文字識別、翻訳システム

In [ ]:
import pytesseract
from PIL import Image
from googletrans import Translator
from google.colab import files
import ipywidgets as widgets
from IPython.display import display

# 対応言語の辞書
LANGUAGES = {
    '英文 (English)': 'en',
    '中文简体 (Simplified Chinese)': 'zh-cn',
    '中文繁体 (Traditional Chinese)': 'zh-tw',
    '日文 (Japanese)': 'ja',
    '韩文 (Korean)': 'ko',
    '法文 (French)': 'fr',
    '德文 (German)': 'de',
    '西班牙文 (Spanish)': 'es'
}

# 画像をアップロード
uploaded = files.upload()

# アップロードされた画像のうち、最初の1枚を使用（複数ある場合は自分で選択も可能）
filename = list(uploaded.keys())[0]
img = Image.open(filename)

# OCR（文字認識）を実行
text = pytesseract.image_to_string(img, lang='eng+chi_sim')
print("\n認識された文字：")
print(text)

# 翻訳処理
if text.strip() == "":
    print("文字が認識されませんでした。")
else:
    # 翻訳先の言語を選ぶためのドロップダウンメニュー
    dropdown = widgets.Dropdown(
        options=LANGUAGES.keys(),
        value='英文 (English)',
        description='翻訳先:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )

    # 翻訳ボタン
    button = widgets.Button(description="翻訳開始", button_style='success')

    # 出力エリア
    output = widgets.Output()

    # 翻訳ボタンがクリックされたときの処理
    def on_translate_clicked(b):
        with output:
            output.clear_output()
            target_lang_code = LANGUAGES[dropdown.value]
            translator = Translator()
            try:
                translated = translator.translate(text, src='auto', dest=target_lang_code)
                print(f"\n{dropdown.value} に翻訳された文字：")
                print(translated.text)
            except Exception as e:
                print(f"翻訳に失敗しました: {e}")

    button.on_click(on_translate_clicked)

    # ウィジェットを表示
    display(dropdown, button, output)


9：HRVに基づくプレシャル判断システム（PYTHONでHRVと呼吸の仮装センサーを構築、数値によって人のプレシャルを判断）

In [ ]:
# 必要であれば、ウィジェットのライブラリをインストール
!pip install ipywidgets --quiet

import ipywidgets as widgets
from IPython.display import display, HTML

# HRV（心拍変動）を模擬するスライダーを作成
sensor_slider_hart = widgets.IntSlider(
    value=100,                # 初期値
    min=0,                    # 最小値
    max=100,                  # 最大値
    step=1,                   # ステップ幅
    description='HRV:',
    style={'description_width': 'initial'},
    continuous_update=True    # 値が変わるたびにイベント発生
)

# 呼吸頻度を模擬するスライダーを作成
sensor_slider_breath = widgets.IntSlider(
    value=30,
    min=0,
    max=100,
    step=1,
    description='呼吸頻度:',
    style={'description_width': 'initial'},
    continuous_update=True
)

# 現在の値を表示するラベル
sensor_label = widgets.Label()

# 警告メッセージを表示するためのHTMLラベル
error_label = widgets.HTML()

# 値が変化したときの処理関数
def update_all(change=None):
    heart = sensor_slider_hart.value       # HRVの値を取得
    breath = sensor_slider_breath.value    # 呼吸頻度の値を取得

    # 現在の値を表示
    sensor_label.value = f"現在のHRV: {heart}、呼吸頻度: {breath}"

    # 条件に応じた警告メッセージの表示
    if heart < 10 and breath > 70:
        error_label.value = "<span style='color:red; font-weight:bold;'>⚠️ プレッシャーが非常に高いです！リラックスしてください</span>"
    elif heart < 20 and breath > 60:
        error_label.value = "<span style='color:red; font-weight:bold;'>⚠️ プレッシャーがやや高いです</span>"
    elif heart < 25 and breath > 50:
        error_label.value = "<span style='color:black; font-weight:bold;'>普通の状態です</span>"
    else:
        error_label.value = "<span style='color:black; font-weight:bold;'>このまま楽しく続けてください</span>"

# スライダーの値が変化したときに update_all を呼び出す
sensor_slider_breath.observe(update_all, names='value')
sensor_slider_hart.observe(update_all, names='value')

# 初期表示
update_all()

# ウィジェットを画面に表示
display(sensor_slider_breath, sensor_slider_hart, sensor_label, error_label)
